In [1]:
# you will be prompted with a window asking to grant permissions
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
# fill in the path in your Google Drive in the string below. Note: do not escape slashes or spaces
import os
datadir = "/content/similarity"
if not os.path.exists(datadir):
  !ln -s "/content/drive/MyDrive/UIUC/14 Sophomore/4th Semester/ATLAS/ROBERTA/roberta" $datadir
os.chdir(datadir)
!pwd

/content/drive/MyDrive/UIUC/14 Sophomore/4th Semester/ATLAS/ROBERTA/roberta


# Testing File

In [8]:
from models.roberta import RoBERTa
import numpy as np
import pandas as pd
import time
import re
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
df = pd.read_csv("course-catalog.csv")
df["Course"] = df['Subject'].astype(str) + " " + df['Number'].astype(str) 
df = df.drop_duplicates(subset=['Subject', 'Number'])
df = df[~df['Section Info'].str.startswith("Same as", na=False)]
df = df[~df["Description"].str.startswith("May be repeated", na=False)]
df = df[~df["Description"].str.startswith("Approved for S/U grading only.", na=False)]
df["Description"] = df["Description"].str.split("Approved for").str[0]
df["Description"] = df["Description"].str.split("May be repeated").str[0]
df["Description"] = df["Description"].str.split("Prerequisite:").str[0]
df["Description"] = df["Description"].str.split("Credit is not").str[0]
df["Description"] = df["Description"].str.split("Continuation of").str[0]
df["Description"] = df["Description"].str.replace(r"\b\d+\s*(or\s*\d+)?\s*(to\s*\d+)?\s*(undergraduate|graduate|professional)\s*hours\b", "", regex=True)
df = df[~df['Description'].str.match(r'^\s*$|^\.*$', na=True)]
df = df.drop_duplicates(subset=["Description"])
df

OSError: [Errno 22] Invalid argument: 'course-catalog.csv'

In [75]:
display(df.head())

,Year,Term,YearTerm,Subject,Number,Name,Description,Credit Hours,Section Info,Degree Attributes,...,Section Status,Enrollment Status,Type,Type Code,Start Time,End Time,Days of Week,Room,Building,Instructors
0,2025,Spring,2025-sp,AAS,100,Intro Asian American Studies,interdisciplinary introduction to the basic co...,3 hours.,NaN,"Social & Beh Sci - Soc Sci, and Cultural Studi...",...,A,Closed,Lecture-Discussion,LCD,02:00 PM,03:20 PM,MW,304,Noyes Laboratory,"Geng, C"
12,2025,Spring,2025-sp,AAS,200,U.S. Race and Empire,invites students to examine histories and narr...,3 hours.,Same as LLS 200.,"Humanities - Hist & Phil, and Cultural Studies...",...,A,CrossListOpen,Lecture-Discussion,LCD,12:30 PM,01:50 PM,TR,148,Armory,"Sawada, E"
15,2025,Spring,2025-sp,AAS,211,Asian Americans and the Arts,examination of asian american artistic express...,3 hours.,"Prerequisite: AAS 100 or AAS 120, or consent o...","Humanities - Lit & Arts, and Cultural Studies ...",...,A,Open,Lecture-Discussion,LCD,03:30 PM,04:50 PM,TR,G18,"Literatures, Cultures, & Ling","Sawada, E"
16,2025,Spring,2025-sp,AAS,215,US Citizenship Comparatively,"examines the racial, gendered, and sexualized ...",3 hours.,"Same as AFRO 215, AIS 295, GWS 215, and LLS 215.","Humanities - Hist & Phil, and Cultural Studies...",...,A,CrossListOpen,Lecture-Discussion,LCD,11:00 AM,12:20 PM,TR,207,Psychology Building,"Siglos, D"
21,2025,Spring,2025-sp,AAS,287,Food and Asian Americans,introduction to the interdisciplinary study of...,3 hours.,"Prerequisite: AAS 100 or AAS 120, or consent o...","Social & Beh Sci - Soc Sci, and Cultural Studi...",...,A,Open,Lecture-Discussion,LCD,02:00 PM,03:20 PM,TR,137,Henry Administration Bldg,"Kang, S"


In [76]:
for value in df["Description"]:
    print(value)

interdisciplinary introduction to the basic concepts and approaches in asian american studies. surveys the various dimensions of asian american experiences including history, social organization, literature, arts, and politics.
invites students to examine histories and narratives of u.s. race and empire, drawing upon multiple theoretical and methodological works in asian american studies and related fields. same as lls 200.
examination of asian american artistic expressions in the visual and the performing arts providing historical, theoretical, and conceptual foundations of understanding the history of various art genres in asian american communities. 
examines the racial, gendered, and sexualized aspects of us citizenship historically and comparatively. interdisciplinary course taught from a humanities perspective. readings draw from critical legal studies, history, literature, literary criticism, and ethnography. same as afro 215, ais 295, gws 215, and lls 215.
introduction to the i

In [63]:
with open("inputcourse.txt", "r") as file:
    file_content1 = file.read()

In [66]:
scores = []
for i in range(len(df)):
    start_time = time.time()
    roberta1 = RoBERTa(file_content1, df.iloc[i]["Description"])
    #print(df.iloc[i]["Subject"], df.iloc[i]["Number"], roberta1.similarity())
    scores.append([df.iloc[i]["Subject"], df.iloc[i]["Number"], roberta1.similarity()])
    print("--- %s seconds ---" % (time.time() - start_time))

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

c:\Users\suvan\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\suvan\.cache\huggingface\hub\models--sentence-transformers--stsb-roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/672 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

ReadTimeout: (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 1ad0f2b4-92cf-4d65-99eb-a40c48983591)')

In [65]:
df_similarity = pd.DataFrame(scores, columns=['Subject', 'Number', 'Similarity'])
df_similarity.sort_values(['Similarity'], ascending=False)

,Subject,Number,Similarity
0,AAS,100,0.357272
1,AAS,211,0.305536
5,ABE,152,0.280696
2,AAS,287,0.263299
3,AAS,589,0.233587
4,ABE,128,0.190305


In [62]:
print(len(df))

3522


In [ ]:
(4046/ 148)

27.33783783783784